In [20]:
import numpy as np
import itk
print("ITK version:" + itk.Version.GetITKVersion())
print(itk.__file__)

ITK version:5.3.0
C:\Dev\ITK-py\Wrapping\Generators\Python\itk\__init__.py


In [21]:
def main_processing(input_image, input_labels, transform_filename, output_image, output_labels):
    print("Read the input image", input_image)
    in_image = itk.imread(input_image)
    print("Read the input labels", input_labels)
    labels = itk.imread(input_labels)
    dimension = in_image.GetImageDimension()
    transforms = itk.transformread(transform_filename)
    direct_transform = transforms[0]
    
    in_image = itk.resample_in_place_image_filter(in_image, rigid_transform=direct_transform)
    labels = itk.resample_in_place_image_filter(labels, rigid_transform=direct_transform)
    
    print("Find the bounding box of the labels")
    bbSO = itk.ImageMaskSpatialObject[dimension].New(labels)
    bbSO.Update()
    
    # this is not the tightest, because BB is computed in index space internally
    bb = bbSO.GetMyBoundingBoxInWorldSpace()
    start = bb.GetMinimum()
    end = bb.GetMaximum()
    Dimension = in_image.GetImageDimension()
    size = [int((end[d] - start[d]) / in_image.GetSpacing()[d]) for d in range(Dimension)]
    
    nearest_interpolator = itk.NearestNeighborInterpolateImageFunction.New(labels)

    labelsAA = itk.resample_image_filter(
        labels,
        interpolator=nearest_interpolator,
        size=size,
        output_spacing=in_image.GetSpacing(),
        output_origin=start,
    )
    
    print("Find the tightest bounding box of the labels")
    bbSO.SetImage(labelsAA);
    bbRegion = bbSO.ComputeMyBoundingBoxInIndexSpace();
    
    labelsAA = itk.region_of_interest_image_filter(
        labelsAA,
        region_of_interest=bbRegion)
    
    # now resample the main image onto the same image grid
    linear_interpolator = itk.LinearInterpolateImageFunction.New(in_image)
    
    out_image = itk.resample_image_filter(
        in_image,
        interpolator=linear_interpolator,
        reference_image=labelsAA,
        use_reference_image=True,
    )
    
    print("Write the axis aligned image", output_image)
    itk.imwrite(out_image, output_image, compression=True)
    print("Write the axis aligned labels", output_labels)
    itk.imwrite(labelsAA, output_labels, compression=True)
    print("All done!")

In [ ]:
# invoke main processing for all the images
image_names = [
    "901-L",
    "901-R",
    "902-L",
    "902-R",
    "906-L",
    "906-R",
    "907-L",
    "907-R",
    "908-L",
    "908-R",
    "915-L",
    "915-R",
    "916-L",
    "916-R",
    "917-L",
    "917-R",
    "918-L",
    "918-R",
    "F9-3wk-01-L",
    "F9-3wk-01-R",
    "F9-3wk-02-L",
    "F9-3wk-02-R",
    "F9-3wk-03-L",
    "F9-3wk-03-R",
    "F9-8wk-01-L",
    "F9-8wk-01-R",
    "F9-8wk-02-L",
    "F9-8wk-02-R",
]

root_dir = "../" # root dir of the repository
for name in image_names:
    main_processing(root_dir+"Data/"+name+".nrrd",
                    root_dir+"Tibias/"+name+"-label.nrrd",
                    root_dir+"Tibias/"+name+"-landmarks.tfm",
                    root_dir+"Tibias/"+name+"-AA.nrrd",
                    root_dir+"Tibias/"+name+"-AA-label.nrrd"
                   )